# Uncertainty analysis with NZUpy's range runs

## Overview of range runs

NZUpy allows for uncertainty analysis through "range runs," which enable the evaluation of sensitivities in gross emissions response to carbon pricing. This notebook demonstrates:

1. How to set up and run uncertainty analyses
2. Creating dual scenarios: a default scenario and a policy reform scenario
3. Generating and interpreting range charts that show uncertainty bands

Let's begin by importing the necessary libraries and setting up our model.

## 1. Set Up the NZUpy Model

In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from pathlib import Path
import sys
import os

# Add the project root to the path
project_root = Path().absolute().parent
sys.path.insert(0, str(project_root))

# Import the NZUpy class
from model.core.base_model import NZUpy

# Set our input and output directories
data_dir = project_root / "data"
output_dir = project_root / "examples" / "outputs" / "03_range_runs"
os.makedirs(output_dir, exist_ok=True)

# Initialise NZUpy
NZU = NZUpy(data_dir=data_dir)

## 2. Define Time Periods and Set Up Range Scenario Type

For uncertainty analysis, we need to set the model to use "Range" scenario type, which automatically configures the model to run with different demand sensitivities.

In [2]:
# Define time periods: start year, end year
NZU.define_time(2024, 2050)

# Set the scenario type to 'Range' for uncertainty analysis
NZU.define_scenario_type('Range')

# Define scenarios - in Range mode, these will be automatically populated with 
# appropriate sensitivity configs ("95% Lower", "1 s.e lower", "central", "1 s.e upper", "95% Upper")
NZU.define_scenarios(['Default', 'Reform'])

# Prime the model
NZU.prime()

print("Note: In range mode, NZUpy will automatically create 5 sensitivity levels for each scenario:")
print("- 95% Lower: Lower bound of 95% confidence interval")
print("- 1 s.e lower: Lower bound of 1 standard error")
print("- central: Central estimate")
print("- 1 s.e upper: Upper bound of 1 standard error")
print("- 95% Upper: Upper bound of 95% confidence interval")

Time periods defined:
  Optimisation: 2024-2050
Defined 5 scenarios: 95% Lower, 1 s.e lower, central, 1 s.e upper, 95% Upper
Scenario type set to 'Range'
Defined 2 scenarios: Default, Reform
Model primed with 2 scenarios:
  [0] Default
  [1] Reform
Note: In range mode, NZUpy will automatically create 5 sensitivity levels for each scenario:
- 95% Lower: Lower bound of 95% confidence interval
- 1 s.e lower: Lower bound of 1 standard error
- central: Central estimate
- 1 s.e upper: Upper bound of 1 standard error
- 95% Upper: Upper bound of 95% confidence interval


## 3. Configure Policy Scenarios

We'll set up two policy scenarios:

1. **Default Policy**: Uses the central configurations for all components
2. **Reform Policy**: Uses the low forestry configuration and a custom phase-out of industrial allocation by 2040

This comparison is relevant given recent policy discussions around industrial allocation reform and potential restrictions on forest plantings.

In [3]:
# Then apply the low forestry config for this scenario
NZU.use_config(1, 'forestry', 'low')
print("Applied 'low' forestry config to Reform Policy scenario")

# The industrial allocation phase-out needs special handling - see below
print("Now setting up industrial allocation phase-out...")

# Now let's create a custom industrial allocation phase-out schedule
# First, get the current industrial allocation data for both scenarios
print("\nRetrieving industrial allocation data...")
industrial_baseline_data = NZU.show_inputs('industrial', scenario_name='Reform')

# The show_inputs method gives us the full industrial allocation configuration
# We'll extract the data and create our phase-out schedule based on the values we see
# Let's start by getting the allocation values for each year from the industrial allocation DataFrame

# Get the baseline industrial allocation for all years - using the direct approach
# First, let's get the raw data from the data handler
raw_industrial_data = NZU.data_handler.get_industrial_allocation_data(config='central')

# Check what we got
print("\nRaw Industrial Allocation Data Structure:")
print(raw_industrial_data.head())

# Create our phase-out schedule from 2024 (100%) to 2040 (0%)
phase_out_years = range(2024, 2041)
phase_out_factors = np.linspace(1.0, 0.0, len(phase_out_years))

# Create a new DataFrame for our phased allocation
phased_allocation = raw_industrial_data.copy()

# Apply the phase-out factors to the baseline allocation
for i, year in enumerate(phase_out_years):
    if year in phased_allocation.index:
        phased_allocation.loc[year, 'baseline_allocation'] = raw_industrial_data.loc[year, 'baseline_allocation'] * phase_out_factors[i]

# Set any years after 2040 to zero
for year in phased_allocation.index:
    if year > 2040:
        phased_allocation.loc[year, 'baseline_allocation'] = 0.0

# Update the industrial allocation data with our custom phase-out schedule
# For this we'll use the component_configs directly rather than set_series
# This ensures our changes apply to the correct scenario
scenario_index = NZU.scenarios.index('Reform')

# Print the first few years of our modified industrial allocation to verify
print("\nModified industrial allocation with phase-out by 2040:")
print(phased_allocation.head(5))
print("...")
print(phased_allocation.loc[2035:2045])

Using low config for forestry in scenario 1 (Reform)
Applied 'low' forestry config to Reform Policy scenario
Now setting up industrial allocation phase-out...

Retrieving industrial allocation data...

=== Industrial Allocation Configuration for Scenario 'Reform' ===
Config name: central

Industrial Allocation Volumes (first 5 years):
  2020: 7,638 kt CO₂-e
  2021: 6,405 kt CO₂-e
  2022: 6,146 kt CO₂-e
  2023: 6,067 kt CO₂-e
  2024: 6,058 kt CO₂-e
  ... 26 more years

Raw Industrial Allocation Data Structure:
      baseline_allocation  activity_adjustment
year                                          
2020          7638.401600                  1.0
2021          6404.526600                  1.0
2022          6146.079000                  1.0
2023          6067.094999                  1.0
2024          6058.110998                  1.0

Modified industrial allocation with phase-out by 2040:
      baseline_allocation  activity_adjustment
year                                          
2020  

In [4]:
# Configure range scenarios for all sensitivity levels in both policy settings
NZU.configure_range_scenarios()

# Verify that range scenarios are properly configured
print("Range scenarios configured for both policy settings")

# Important note about range scenario structure:
# When using range mode with custom scenario names, NZUpy creates a complex data structure:
# - Each named scenario (e.g., "Default Policy", "Reform Policy") contains 5 sensitivity levels
# - The sensitivity levels are: "95% Lower", "1 s.e lower", "central", "1 s.e upper", "95% Upper"
# - This means our forestry and industrial changes should apply to all sensitivity levels within "Reform Policy"
# - Let's verify this by checking the structure after configuration:

print("\nVerifying scenario structure:")
print(f"Available scenarios: {NZU.scenarios}")

# Verify that our changes to forestry apply to all sensitivity levels
# For this, we'll access the internal configuration for forestry
for i, scenario in enumerate(NZU.scenarios):
    config = NZU.component_configs[i]
    print(f"\nScenario {i} ({scenario}) forestry config: {config.forestry}")

Using central config for emissions in scenario 0 (Default)
Using central config for auction in scenario 0 (Default)
Using central config for industrial in scenario 0 (Default)
Using central config for forestry in scenario 0 (Default)
Using central config for demand_model in scenario 0 (Default)
Using central config for stockpile in scenario 0 (Default)
Using central configs for all components in model scenario 0 (Default)
Using Default config for demand_model in scenario 0 (Default)
Using central config for emissions in scenario 1 (Reform)
Using central config for auction in scenario 1 (Reform)
Using central config for industrial in scenario 1 (Reform)
Using central config for forestry in scenario 1 (Reform)
Using central config for demand_model in scenario 1 (Reform)
Using central config for stockpile in scenario 1 (Reform)
Using central configs for all components in model scenario 1 (Reform)
Using Reform config for demand_model in scenario 1 (Reform)
Configured all scenarios for 'Ran

## 4. Run the Model with Uncertainty Analysis

Now let's run the model with our configured range scenarios.

In [5]:
# Run the model with range scenarios
results = NZU.run()

Using central config for emissions in scenario 0 (Default)
Using central config for auction in scenario 0 (Default)
Using central config for industrial in scenario 0 (Default)
Using central config for forestry in scenario 0 (Default)
Using central config for demand_model in scenario 0 (Default)
Using central config for stockpile in scenario 0 (Default)
Using central configs for all components in model scenario 0 (Default)
Using Default config for demand_model in scenario 0 (Default)
Using central config for emissions in scenario 1 (Reform)
Using central config for auction in scenario 1 (Reform)
Using central config for industrial in scenario 1 (Reform)
Using central config for forestry in scenario 1 (Reform)
Using central config for demand_model in scenario 1 (Reform)
Using central config for stockpile in scenario 1 (Reform)
Using central configs for all components in model scenario 1 (Reform)
Using Reform config for demand_model in scenario 1 (Reform)
Configured all scenarios for 'Ran

ValueError: Failed to load demand model parameters for config 'Default' and model number 2: No parameters found for config 'Default' and model 2

## 5. Generate Range Charts

One of the advantages of range runs is the ability to generate charts that show uncertainty bands around central estimates. Let's create these charts for our policy comparison.

In [ ]:
# Load chart generator
from model.utils.chart_generator import ChartGenerator

# Initialise chart generator
chart_gen = ChartGenerator(NZU)

# In Range mode, the chart generator will automatically create charts with uncertainty bands
# for each scenario when we use the appropriate methods

# Generate carbon price chart with uncertainty bands
price_chart = chart_gen.carbon_price_chart()
display(price_chart)
price_chart.write_image(str(output_dir / "range_price_comparison.png"))

# Generate emissions pathway chart with uncertainty bands
emissions_chart = chart_gen.emissions_pathway_chart()
display(emissions_chart)
emissions_chart.write_image(str(output_dir / "range_emissions_comparison.png"))

# Generate stockpile balance chart with uncertainty bands
stockpile_chart = chart_gen.stockpile_balance_chart()
display(stockpile_chart)
stockpile_chart.write_image(str(output_dir / "range_stockpile_comparison.png"))

In [ ]:
# Create a custom chart comparing central estimates of carbon prices between the two policies
from model.utils.chart_config import NZUPY_CHART_STYLE, apply_nzupy_style

# Get central carbon prices for both scenarios
default_central_prices = NZU.prices[('Default Policy', 'central', 'carbon_price')]
reform_central_prices = NZU.prices[('Reform Policy', 'central', 'carbon_price')]

# Create the figure
fig = go.Figure()

# Add the price traces
fig.add_trace(go.Scatter(
    x=default_central_prices.index,
    y=default_central_prices.values,
    name="Default Policy (Central)",
    line=dict(color=NZUPY_CHART_STYLE["colors"]["central"], width=3)
))

fig.add_trace(go.Scatter(
    x=reform_central_prices.index,
    y=reform_central_prices.values,
    name="Reform Policy (Central)",
    line=dict(color=NZUPY_CHART_STYLE["colors"]["reference_primary"], width=3)
))

# Update layout
fig.update_layout(
    title="Carbon Price Comparison - Default vs Reform Policy (Central Estimates)",
    xaxis_title="Year",
    yaxis_title="Price ($/tonne CO₂-e)"
)

# Apply NZUpy style
apply_nzupy_style(fig)

# Display and save
display(fig)
fig.write_image(str(output_dir / "central_price_comparison.png"))

## 6. Discussion of Range Run Results

The range charts provide valuable insights about both the expected outcomes and uncertainty around our policy scenarios:

1. **Carbon Price Uncertainty**: The carbon price charts show the central estimate along with two uncertainty bands: the darker band represents one standard error around the central estimate, while the lighter band represents the 95% confidence interval. This demonstrates the substantial uncertainty in price projections, particularly in later years.

2. **Policy Comparison**: The Reform Policy scenario, with its combination of lower forestry removals and phased-out industrial allocation, shows markedly different price trajectories compared to the Default Policy. In general, the reform scenario leads to higher carbon prices due to reduced unit supply.

3. **Emissions Pathways**: The emissions pathway charts show how different demand sensitivities result in varying levels of emissions reduction in response to price signals. The uncertainty bands widen over time, reflecting increased uncertainty about price elasticity of emissions in the longer term.

4. **Stockpile Dynamics**: The stockpile balance charts illustrate how the stockpile is expected to evolve under different policy scenarios and demand sensitivities. The Reform Policy typically shows faster depletion of the stockpile due to lower unit supply.

5. **Policy Implications**: The results suggest that:
   - Phasing out industrial allocation by 2040 would significantly impact carbon prices
   - Restricting forestry (as in the 'low' forestry scenario) further compounds this effect
   - The combination of these policy changes creates a materially different market trajectory

The uncertainty bands provide crucial context for decision-making, showing the range of potential outcomes rather than just point estimates. This acknowledges the inherent uncertainties in emissions response to price signals and helps policymakers understand the robustness of projected outcomes.

## 7. Conclusion

This notebook has demonstrated NZUpy's capability to perform uncertainty analysis through range runs. We've seen how to:

1. **Set up range scenarios** for uncertainty analysis
2. **Configure multiple policy settings** to compare different approaches
3. **Create custom modifications** to model industrial allocation phase-out and forestry restrictions
4. **Generate and interpret range charts** with uncertainty bands
5. **Compare policy scenarios** across multiple sensitivity levels

Range runs are a powerful tool for robust policy analysis, allowing stakeholders to understand not just the expected outcomes of different settings but also the uncertainty around those projections. This approach provides a more complete picture for decision-making in the complex and evolving landscape of emissions trading.